In [ ]:
!pip install opacus

In [ ]:
!wget "https://raw.githubusercontent.com/mbilal0612/medical-data/main/medical_data.txt"

--2024-05-28 11:55:35--  https://raw.githubusercontent.com/mbilal0612/medical-data/main/medical_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37395 (37K) [text/plain]
Saving to: ‘medical_data.txt’

medical_data.txt    100%[===================>]  36.52K  --.-KB/s    in 0.02s   

2024-05-28 11:55:35 (2.35 MB/s) - ‘medical_data.txt’ saved [37395/37395]



In [ ]:
import json
!git clone https://github.com/mbilal0612/medical-data.git

Cloning into 'medical-data'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 10 (delta 1), reused 10 (delta 1), pack-reused 0
Receiving objects: 100% (10/10), 5.82 MiB | 8.38 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:

# Read data from the input JSON file
with open('medical-data/sources/medical_dataset_train.json', 'r') as file:
    data = json.load(file)

input_data = [entry['input'] + '\n'  + '\n'  + entry['answer_chatdoctor'] +'\n' +'\n'    for entry in data]

# Write the combined data to a new JSON file
with open('medical_data2.txt', 'w') as file:
    file.writelines(input_data)

print("Data has been combined and written to 'medical_data2.txt'")


Data has been combined and written to 'medical_data2.txt'


In [ ]:
# Read data from the input JSON file
with open('medical-data/sources/medical_dataset_train.json', 'r') as file:
    data = json.load(file)

input_data = [entry['input'] + '\n'  + entry['answer_chatdoctor'] +'\n' + '---'  +'\n' for entry in data]

# Write the combined data to a new JSON file
with open('medical_data_text.txt', 'w') as file:
    file.writelines(input_data)

print("Data has been combined and written to 'medical_data2.txt'")

Data has been combined and written to 'medical_data2.txt'


# Imports

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup

from opacus import PrivacyEngine
from opacus.utils.batch_memory_manager import BatchMemoryManager
from opacus.data_loader import DPDataLoader
from opacus.accountants import RDPAccountant


# TextDataset class
#### use to process the dataset


In [ ]:

import torch
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, file_path, tokenizer, block_size=512):
        self.tokenizer = tokenizer
        self.block_size = min(block_size, tokenizer.model_max_length)  # Ensure block size does not exceed model's max length
        self.examples = []

        with open(file_path, 'r', encoding='utf-8') as f:
            current_chunk = []
            for line in f:
                if line.strip() == '---':
                    if current_chunk:  # Only tokenize and add non-empty chunks
                        tokenized_chunk = tokenizer.encode(' '.join(current_chunk))
                        for i in range(0, len(tokenized_chunk), self.block_size):
                            self.examples.append(tokenized_chunk[i:i + self.block_size])  # Append tokenized chunks
                    current_chunk = []  # Reset chunk
                else:
                    current_chunk.append(line.strip())

            # Tokenize and add the last chunk
            if current_chunk:
                tokenized_chunk = tokenizer.encode(' '.join(current_chunk))
                for i in range(0, len(tokenized_chunk), self.block_size):
                    self.examples.append(tokenized_chunk[i:i + self.block_size])

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        # Convert the list to a tensor
        sequence_tensor = torch.tensor(self.examples[i], dtype=torch.long)
        # Pad sequences to the same length
        padded_sequence = torch.nn.functional.pad(sequence_tensor, (0, self.block_size - len(self.examples[i])), value=self.tokenizer.pad_token_id)
        return padded_sequence

# class TextDataset(Dataset):
#     def __init__(self, file_path, tokenizer, block_size=512):
#         self.tokenizer = tokenizer
#         self.block_size = min(block_size, tokenizer.model_max_length)  # Ensure block size does not exceed model's max length
#         self.examples = []

#         with open(file_path, 'r', encoding='utf-8') as f:
#             current_chunk = []
#             for line in f:
#                 if line.strip() == '---':
#                     if current_chunk:  # Only tokenize and add non-empty chunks
#                         tokenized_chunk = tokenizer.encode(' '.join(current_chunk))
#                         for i in range(0, len(tokenized_chunk), self.block_size):
#                             self.examples.append(tokenized_chunk[i:i + self.block_size])  # Append tokenized chunks
#                     current_chunk = []  # Reset chunk
#                 else:
#                     current_chunk.append(line.strip())

#             # Tokenize and add the last chunk
#             if current_chunk:
#                 tokenized_chunk = tokenizer.encode(' '.join(current_chunk))
#                 for i in range(0, len(tokenized_chunk), self.block_size):
#                     self.examples.append(tokenized_chunk[i:i + self.block_size])

#     def __len__(self):
#         return len(self.examples)

#     def __getitem__(self, i):
#         # Pad sequences to the same length
#         padded_sequence = torch.nn.functional.pad(self.examples[i], (0, self.block_size - len(self.examples[i])), value=self.tokenizer.pad_token_id)
#         return padded_sequence



# class TextDataset(Dataset):
#     def __init__(self, file_path, tokenizer, block_size=512):
#         self.tokenizer = tokenizer
#         self.block_size = min(block_size, tokenizer.model_max_length)  # Ensure block size does not exceed model's max length
#         self.examples = []

#         with open(file_path, 'r', encoding='utf-8') as f:
#             current_chunk = []
#             for line in f:
#                 if line.strip() == '---':
#                     if current_chunk:  # Only tokenize and add non-empty chunks
#                         tokenized_chunk = tokenizer.encode(' '.join(current_chunk))
#                         for i in range(0, len(tokenized_chunk), self.block_size):
#                             self.examples.append(tokenized_chunk[i:i + self.block_size])  # Append tokenized chunks
#                     current_chunk = []  # Reset chunk
#                 else:
#                     current_chunk.append(line.strip())

#             # Tokenize and add the last chunk
#             if current_chunk:
#                 tokenized_chunk = tokenizer.encode(' '.join(current_chunk))
#                 for i in range(0, len(tokenized_chunk), self.block_size):
#                     self.examples.append(tokenized_chunk[i:i + self.block_size])

#     def __len__(self):
#         return len(self.examples)

#     def __getitem__(self, i):
#         return torch.tensor(self.examples[i], dtype=torch.long)

# Upload medical_data.txt to session storage before running the next cell

In [ ]:
# Dataset class
# class TextDataset(Dataset):
#     def __init__(self, file_path, tokenizer, block_size=512):
#         self.tokenizer = tokenizer
#         self.block_size = block_size
#         self.examples = []

#         with open(file_path, 'r', encoding='utf-8') as f:
#             text = f.read()

#         tokenized_text = tokenizer.encode(text)
#         for i in range(0, len(tokenized_text) - block_size + 1, block_size):
#             self.examples.append(tokenized_text[i:i + block_size])

#     def __len__(self):
#         return len(self.examples)

#     def __getitem__(self, i):
#         return torch.tensor(self.examples[i])

# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Create the dataset
dataset = TextDataset('medical_data.txt', tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Initialize the model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training parameters
epochs = 3
learning_rate = 5e-5
max_grad_norm = 1.0
sigma = 1.0
sample_rate = 0.01

# Optimizer and learning rate scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=len(dataloader) * epochs)

# Privacy engine
privacy_engine = PrivacyEngine(
    model,
    sample_rate=sample_rate,
    noise_multiplier=sigma,
    max_grad_norm=max_grad_norm,
    accountant="rdp"
)
privacy_engine.attach(optimizer)

# Training loop with differential privacy
model.train()

for epoch in range(epochs):
    for batch in dataloader:
        batch = batch.to(device)

        optimizer.zero_grad()
        outputs = model(batch, labels=batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        scheduler.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")

print("Training complete.")


Token indices sequence length is longer than the specified maximum sequence length for this model (9906 > 1024). Running this sequence through the model will result in indexing errors


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

TypeError: PrivacyEngine.__init__() got an unexpected keyword argument 'sample_rate'

# **Without Differential Privacy**

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Create the dataset
dataset = TextDataset('medical_data_text.txt', tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Initialize the model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training parameters
epochs = 3
learning_rate = 5e-4
warmup_steps = 100

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=len(dataloader) * epochs)

# Training loop
model.train()

# for epoch in range(epochs):
#     for batch in dataloader:
#         # Tokenize sequences with the same tokenizer settings
#         tokenized_batch = [tokenizer.encode(seq) for seq in batch]

#         # Pad sequences in the batch
#         padded_batch = pad_sequence(tokenized_batch, batch_first=True, padding_value=tokenizer.pad_token_id).to(device)

#         outputs = model(padded_batch, labels=padded_batch)
#         loss = outputs.loss

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         scheduler.step()

#         print(f"Epoch: {epoch}, Loss: {loss.item()}")

# print("Training complete.")

for epoch in range(epochs):
    for batch in dataloader:
        batch = batch.to(device)
        outputs = model(batch, labels=batch)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")

print("Training complete.")

# Save the trained model and tokenizer
output_model_dir = 'trained_models/non_private_medical_gpt/'  # Specify your desired directory to save the trained model and tokenizer

model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)

print("Model and tokenizer saved.")


NameError: name 'GPT2Tokenizer' is not defined

# Getting a response the model



In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the trained tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('trained_models/non_private_medical_gpt/')
model = GPT2LMHeadModel.from_pretrained('trained_models/non_private_medical_gpt/')

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Number of parameters in the model: {num_params/10**6}M")

# Set the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Provide context (including your question)
context = """
I have a fever and a runny nose.
"""

# Tokenize the context
input_ids = tokenizer.encode(context, return_tensors='pt').to(device)

# Generate text based on the context
output = model.generate(input_ids, max_length=75, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

# Decode the output tokens back into text
response = tokenizer.decode(output[0], skip_special_tokens=True)

print("Model's Response:")
print(response)

Number of parameters in the model: 124.439808M
Model's Response:

I have a fever and a runny nose.
 Disease: Celiac Disease **Symptoms:** Diarrhea, bloating, gas, fatigue, weight loss, anemia.  Good afternoon. What brings you in today?  I've been having a lot of digestive issues like diarrhea and bloating.  That sounds uncomfortable. Do you also experience gas


# Dialogue Dataset

##### So after just throwing data at the problem now we'll do some better data preprocessing so that the transformer learns better and is able to differentiate patient from

In [ ]:
from torch.utils.data import Dataset

class DialogueDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.dialogues = []
        with open(file_path, 'r', encoding='utf-8') as f:
            dialogue = []
            for line in f:
                line = line.strip()
                if line:
                    dialogue.append(line)
                else:
                    if len(dialogue) == 2:  # Ensure each dialogue has exactly two utterances (doctor and patient)
                        # Tokenize doctor and patient utterances
                        tokenized_doctor = self.tokenizer.encode(dialogue[0], add_special_tokens=False)
                        tokenized_patient = self.tokenizer.encode(dialogue[1], add_special_tokens=False)

                        # Check if tokenization was successful
                        if tokenized_doctor is not None and tokenized_patient is not None:
                            # Combine tokens and add special tokens
                            input_ids = [self.tokenizer.cls_token_id] + tokenized_doctor + [self.tokenizer.sep_token_id] + tokenized_patient + [self.tokenizer.sep_token_id]

                            # Truncate or pad input sequence to max_length
                            input_ids = input_ids[:self.max_length]
                            padding_length = self.max_length - len(input_ids)
                            input_ids += [self.tokenizer.pad_token_id] * padding_length

                            attention_mask = [1] * len(input_ids)

                            self.dialogues.append({
                                'input_ids': torch.tensor(input_ids),
                                'attention_mask': torch.tensor(attention_mask)
                            })

                        dialogue = []  # Reset dialogue for the next pair

    def __len__(self):
        return len(self.dialogues)

    def __getitem__(self, idx):
        return self.dialogues[idx]


In [ ]:
# Initialize the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Create the dataset
dataset = DialogueDataset('medical_data2.txt', tokenizer)

# Create the data loader
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Training parameters
epochs = 120
learning_rate = 5e-4
warmup_steps = 100

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=len(dataloader) * epochs)

# Training loop
model.train()

for epoch in range(epochs):
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")

print("Training complete.")

# Save the trained model and tokenizer
output_model_dir = 'trained_models/doctor_patient_gpt/'
model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)

print("Model and tokenizer saved.")

RuntimeError: Could not infer dtype of NoneType